### LSB index 
## AMIE manus period 

period : 1 Oct 2011 - 31 March 2012


In [1]:
import glob 
import pandas as pd

import xarray as xr

In [2]:
from dask_jobqueue import SLURMCluster
import matplotlib.pyplot as plt 
import glob
import os
import xarray as xr 
import pandas as pd
import numpy as np


from dask_jobqueue import SLURMCluster

cluster = SLURMCluster(
    job_name="Climt1",          # --job-name
    cores=12,                     # Number of cores per task (adjust if needed)
    processes=6,                 # One process per task
    memory="100GB",               # --mem
    walltime="01:15:00",         # --time
    queue="med",               # --partition
    log_directory=".",           # Logs will be saved to the current directory
)


from dask.distributed import Client


cluster.scale(jobs=10)
client = Client(cluster)
client

/home1/nalex2023/.conda/envs/nma/lib/python3.13/site-packages/distributed/node.py:187: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 39857 instead
  warnings.warn(


Connection method: Cluster object,Cluster type: dask_jobqueue.SLURMCluster
Dashboard: http://10.42.239.61:39857/status,
Dashboard: http://10.42.239.61:39857/status,Workers: 0
Total threads: 0,Total memory: 0 B
Comm: tcp://10.42.239.61:45453,Workers: 0
Dashboard: http://10.42.239.61:39857/status,Total threads: 0
Started: Just now,Total memory: 0 B


In [24]:
era_data = glob.glob('/home1/nalex2023/Datasets/era5_manus/*.nc')
dft_era = pd.DataFrame(era_data, columns=['files'])


dft_era['year'] = dft_era['files'].str.split('_').str[-1].str[:4]
dft_era['month'] = dft_era['files'].str.split('_').str[-1].str[4:6].astype(int)



In [25]:
amie_period = pd.date_range(start='2011-10-01', end='2012-03-31', freq='D')

dft_era_amie = dft_era[(dft_era['year'].isin(amie_period.year.astype(str))) & (dft_era['month'].isin(amie_period.month))]


era_dset_amie = xr.open_mfdataset(dft_era_amie['files'].values, combine='by_coords')    

In [ ]:
#out_amie = '/home1/nalex2023/Datasets/LSB_OUTS/amie_period/'




